In [1]:
# 2019 Q2
# Dedup with email only
import pandas as pd
import numpy as np
import datetime
import os
import gc
import glob
os.getcwd()

'/home/jian/Projects/Big_Lots/Live_Ramp/Mapping_Files/2019_Q2'

In [2]:
def recursive_file_gen(my_root_dir):
    for root, dirs, files in os.walk(my_root_dir):
        for file in files:
            yield os.path.join(root, file)

In [3]:
'''
Segments:
1. Quarterly regular
2. New SignUps
3. 5 Market SOTF Audience
4. NewSigns & No Purchase
5. Experian
'''

'\nSegments:\n1. Quarterly regular\n2. New SignUps\n3. 5 Market SOTF Audience\n4. NewSigns & No Purchase\n5. Experian\n'

In [4]:
seg_1=list(recursive_file_gen("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q2/output_final_20190422/Test_and_Control/uploaded/"))
print("seg_1: "+str(len(seg_1)))

seg_2=list(recursive_file_gen("/home/jian/celery/Append_to_celery_LiveRamp/monthly_update/"))
seg_2=[x for x in seg_2 if ".csv" in x]
seg_2=pd.DataFrame({"file_path":seg_2})
seg_2['date']=seg_2['file_path'].apply(lambda x: x.split("onthly_update/output_")[1][:10])
seg_2=seg_2[seg_2['date']>="2019-04-15"] # 1 wave ahead of the quarter
seg_2=seg_2[seg_2['date']<"2019-08-03"] # 
seg_2=seg_2['file_path'].tolist()
print("seg_2: "+str(len(seg_2)))

seg_3=glob.glob("/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Planner_Requests/audience_Kaylar_SOTF_markets_20190708/output_2019-07-10/*.csv")
print("seg_3: "+str(len(seg_3)))

seg_4=glob.glob("/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Planner_Requests/Past_5_quarters_new_sign_up_no_purchase/*.csv")
print("seg_4: "+str(len(seg_4)))

seg_5=glob.glob("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q2/Experian_Email_Openers_Group/output_2nd_upload/*.csv")
seg_5=[x for x in seg_5 if "summary" not in x]
print("seg_5: "+str(len(seg_5)))

seg_1: 50
seg_2: 24
seg_3: 5
seg_4: 5
seg_5: 20


In [5]:
all_file_list=seg_1+seg_2+seg_3+seg_4+seg_5
len(all_file_list)

104

In [7]:
df_output_all_Q2_seg=pd.DataFrame()
col_list=[]
for file in all_file_list:
    try:
        df=pd.read_csv(file,dtype=str,usecols=['customer_id_hashed','email_address_hash','segment'])
    except:
        df=pd.read_csv(file,dtype=str,usecols=['customer_id_hashed','Email_Address_hash','segment'])
        df=df.rename(columns={"Email_Address_hash":"email_address_hash"})
        
    df_output_all_Q2_seg=df_output_all_Q2_seg.append(df)

In [8]:
df_output_all_Q2_seg.shape,df_output_all_Q2_seg['email_address_hash'].nunique()

((33928202, 3), 25762231)

In [11]:
df_output_all_Q2_seg=df_output_all_Q2_seg.drop_duplicates("email_address_hash")

In [12]:
df_output_all_Q2_seg.to_csv("/home/jian/Projects/Big_Lots/Live_Ramp/Mapping_Files/2019_Q2/BL_mapping_file_2019Q2_JL_"+str(datetime.datetime.now().date())+".csv",index=False)


In [13]:
import paramiko

In [14]:
host = "64.237.51.251" #hard-coded
port = 22
transport = paramiko.Transport((host, port))

password = "jian@juba2017" #hard-coded
username = "jian" #hard-coded
transport.connect(username = username, password = password)
sftp = paramiko.SFTPClient.from_transport(transport)


In [15]:
local_path="/home/jian/Projects/Big_Lots/Live_Ramp/Mapping_Files/2019_Q2/BL_mapping_file_2019Q2_JL_2019-09-23.csv"

In [16]:
remote_folder='/mnt/drv5/jian/Big_Lots/BL_1_to_1_2019Q2/Q2_mapping_file/'
basename=os.path.basename(local_path)
remote_path=remote_folder+basename
sftp.put(local_path,remote_path)

sftp.close()
remote_path.close()

AttributeError: 'str' object has no attribute 'close'

In [17]:
transport.close()